# Version 1 of the final project 

## Preprocessing 

In [1]:
#Imports necessary libraries 
import pandas as pd
import numpy as np
import networkx as nx
import json as js
import requests
import os

In [3]:
data = js.load(open('tweet_jsons/response1.json'))
tweet_df = pd.DataFrame(data["data"])

for i in range(2, 63):
    filepath = f'tweet_jsons/response{i}.json'
    data = js.load(open(filepath))
    new_df = pd.DataFrame(data["data"])
    tweet_df = pd.concat([new_df, tweet_df])

print(len(tweet_df)) # =3500 Tweets currently
print(len(tweet_df['author_id'].unique())) # =3046 users currently

# to-do: 
# pull corresponding author information from separate api endpoint for all unique author ids

3501
3046


In [4]:
tweet_df.head()

,author_id,context_annotations,conversation_id,created_at,edit_history_tweet_ids,entities,id,in_reply_to_user_id,lang,public_metrics,referenced_tweets,reply_settings,source,text,geo
0,1229060372105134080,[{'domain': {'description': 'Real world events...,1324865785513332738,2020-11-07T00:09:58.000Z,[1324866646834606081],"{'annotations': [{'end': 101, 'normalized_text...",1324866646834606081,1236085967666192385,en,"{'like_count': 0, 'quote_count': 0, 'reply_cou...","[{'id': '1324866614014210050', 'type': 'replie...",everyone,Twitter Web App,@JavoeGator @JonahLandergan The voting machine...,NaN
1,1229060372105134080,[{'domain': {'description': 'Real world events...,1324865385448042496,2020-11-07T00:09:32.000Z,[1324866538059558913],"{'annotations': [{'end': 100, 'normalized_text...",1324866538059558913,1270762579774902272,en,"{'like_count': 1, 'quote_count': 0, 'reply_cou...","[{'id': '1324866484536012800', 'type': 'replie...",everyone,Twitter Web App,@hannahyusuck @saintherons The voting machines...,NaN
2,1229060372105134080,[{'domain': {'description': 'Real world events...,1324866302100508676,2020-11-07T00:08:55.000Z,[1324866382966775810],"{'annotations': [{'end': 88, 'normalized_text'...",1324866382966775810,1661961012,en,"{'like_count': 0, 'quote_count': 0, 'reply_cou...","[{'id': '1324866302100508676', 'type': 'replie...",everyone,Twitter Web App,@FoxFyrePhotos The voting machines related to ...,NaN
3,1229060372105134080,[{'domain': {'description': 'Real world events...,1324865813325680641,2020-11-07T00:08:40.000Z,[1324866319896940544],"{'annotations': [{'end': 85, 'normalized_text'...",1324866319896940544,1314213370434744326,en,"{'like_count': 0, 'quote_count': 0, 'reply_cou...","[{'id': '1324865813325680641', 'type': 'replie...",everyone,Twitter Web App,@bertzbeess The voting machines related to the...,NaN
4,1229060372105134080,[{'domain': {'description': 'Named people in t...,1324835859812798465,2020-11-07T00:08:27.000Z,[1324866264913899522],"{'annotations': [{'end': 98, 'normalized_text'...",1324866264913899522,20939466,en,"{'like_count': 0, 'quote_count': 0, 'reply_cou...","[{'id': '1324866011049435137', 'type': 'replie...",everyone,Twitter Web App,@rajesh06 @NateSilver538 The voting machines r...,NaN


In [5]:
data = js.load(open('user_jsons/response_user0.json'))
user_df = pd.DataFrame(data["data"])

for i in range(1, 3501):
    if i not in [650, 746, 766, 1415, 1527, 2858, 2878, 3312, 3433]:
        filepath = f'user_jsons/response_user{i}.json'
        data = js.load(open(filepath))
        new_df = pd.DataFrame(data["data"])
        user_df = pd.concat([new_df, user_df])

In [6]:
df2 = pd.json_normalize(user_df['public_metrics'])

In [7]:
df2

,followers_count,following_count,listed_count,tweet_count
0,12,230,0,2848
1,1179,899,15,219404
2,465,901,0,18778
3,78,1029,0,3189
4,2592,4951,15,54783
...,...,...,...,...
3487,90,477,1,6549
3488,90,477,1,6549
3489,90,477,1,6549
3490,90,477,1,6549


In [8]:
user_df = user_df.reset_index()

In [9]:
user_df = user_df.join(df2)

In [10]:
user_df = user_df.drop(columns={'public_metrics'})

In [18]:
user_df

,index,created_at,description,id,name,username,location,entities,followers_count,following_count,listed_count,tweet_count
0,0,2019-09-15T14:19:50.000Z,Conservative Republican who respects GOD and t...,1173239803128582144,Rebecca Vargo,vargo_rebecca,NaN,NaN,12,230,0,2848
1,0,2012-12-26T20:19:10.000Z,,1037981756,Carey Lauren,CareyLauren334,United States,NaN,1179,899,15,219404
2,0,2019-07-31T15:39:01.000Z,sta fungia di minghia...🇺🇸MAGA🇺🇸,1156590095622922241,Keep Your Hands Off Our Constitution,KeepYourHandsO2,NaN,NaN,465,901,0,18778
3,0,2009-10-23T13:39:35.000Z,,84592949,HOMESWEETHOME,FlaDebbie,"Madeira Beach, FL",NaN,78,1029,0,3189
4,0,2008-08-09T23:57:26.000Z,There’s nobody that’s coming to save us. You a...,15793556,Daniel Harris - Ballot Harvester 2024,DanielLHarrisUS,NaN,NaN,2592,4951,15,54783
...,...,...,...,...,...,...,...,...,...,...,...,...
3487,0,2020-02-16T15:10:10.000Z,,1229060372105134080,Mr. Gekko,sicilianslice12,NaN,NaN,90,477,1,6549
3488,0,2020-02-16T15:10:10.000Z,,1229060372105134080,Mr. Gekko,sicilianslice12,NaN,NaN,90,477,1,6549
3489,0,2020-02-16T15:10:10.000Z,,1229060372105134080,Mr. Gekko,sicilianslice12,NaN,NaN,90,477,1,6549
3490,0,2020-02-16T15:10:10.000Z,,1229060372105134080,Mr. Gekko,sicilianslice12,NaN,NaN,90,477,1,6549


In [19]:
#user_df['entities']
test= pd.json_normalize(user_df['entities'])

In [20]:
test

,description.mentions,description.hashtags,url.urls,description.urls,description.cashtags
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
3487,NaN,NaN,NaN,NaN,NaN
3488,NaN,NaN,NaN,NaN,NaN
3489,NaN,NaN,NaN,NaN,NaN
3490,NaN,NaN,NaN,NaN,NaN


In [ ]:
val = test['description.hashtags'][3]
val

In [ ]:
def create_tag_list(row):
    tag_list = []
    if type(row['description.hashtags'])!=float:
        for tag_dict in row['description.hashtags']:
            tag_list.append(tag_dict['tag'])
        return tag_list
    else:
        return None

In [ ]:
test['hashtags'] = test.apply(lambda row: create_tag_list(row), axis=1)

In [ ]:
user_df = user_df.join(test['hashtags'])

In [ ]:
user_df = user_df.drop(columns='entities')

In [ ]:
user_df = user_df.drop(columns='hashtags')

In [ ]:
user_df.columns

In [ ]:
tweet_df.columns

In [ ]:
tweet_fields = ["in_reply_to_user_id", "text"]
user_fields = ["created_at", "id", "user_id", "followers_count", "name"]
dont_have = ['in_reply_to_screen_name', "in_reply_to_status_id", "retweeted_id",
"retweeted_screen_name", "user_mentions_screen_name", "user_mentions_id"]
# note screen_name is name

In [ ]:
tweet_types = pd.json_normalize(tweet_df['referenced_tweets'])

In [ ]:
tweet_df.loc[tweet_df['id']=='1324866646834606081']['referenced_tweets']

In [ ]:
def create_referenced_list(row):
    tag_overall_dict = {}
    if type(row['referenced_tweets'])!=float:
        for tag_dict in row['referenced_tweets']:
            tag_overall_dict[tag_dict['type']] = tag_dict['id']
        return tag_overall_dict
    else:
        return None

In [ ]:
test_df = tweet_df.copy()

In [ ]:
test_df['testdict'] = tweet_df.apply(lambda row: create_referenced_list(row), axis=1)

In [ ]:
for dic in test_df['testdict']:
    print(dic['replied_to'])
    print('replied_to' in dic)
    break

In [ ]:
reply_testing =  pd.DataFrame(columns=['retweeted', 'replied_to', 'quoted'])
def process_row(row):
    if row['testdict'] is None:
        to_append = [None, None, None] 
    elif 'replied_to' in row['testdict']:
        to_append = [row['testdict']['replied_to'], None, None]
    elif 'retweeted' in row['testdict']:
        to_append = [None, row['testdict']['retweeted'], None]
    elif 'quoted' in row['testdict']:
        to_append = [None, None, row['testdict']['quoted']]
    reply_testing.loc[len(reply_testing.index)] = to_append

In [ ]:
test_df.apply(lambda row: process_row(row), axis=1)

In [ ]:
test_df = test_df.join(reply_testing)

In [ ]:
tweet_df = test_df

In [ ]:
tweet_df = tweet_df.drop(columns='testdict')

In [ ]:
def pop_public_metrics(row):
    

In [ ]:
pub_metrics = pd.json_normalize(tweet_df['public_metrics'])

In [ ]:
tweet_df = tweet_df.join(pub_metrics)

In [ ]:
tweet_df = tweet_df.drop(columns=['public_metrics', 'referenced_tweets'])

In [ ]:
entity_df = pd.json_normalize(tweet_df['entities'])

In [ ]:
entity_df

In [ ]:
mentions_df = pd.DataFrame(columns=['user_mentioned'])
def process_mentions(mentions):
    if mentions is None or type(mentions)==float:
        to_append = [None]
    else:
        for mentions_dict in mentions:
            if mentions_dict is None:
                to_append = [None]
            else:
                to_append = [mentions_dict['id']]
    mentions_df.loc[len(mentions_df.index)] = to_append

In [ ]:
annotations_df = pd.DataFrame(columns=['Place', 'Organization', 'Person', 'Other', 'Probability'])
def process_annotations(annotations):
    if annotations is None or type(annotations)==float:
        to_append = [None, None, None, None, None]
    else:
        for an_dict in annotations:
            if an_dict is None:
                to_append = [None, None, None, None, None]
            if an_dict['type'] == 'Place':
                to_append = [an_dict['normalized_text'], None, None, None, an_dict['probability']]
            elif an_dict['type'] == 'Organization':
                to_append = [None, an_dict['normalized_text'], None, None, an_dict['probability']]
            elif an_dict['type'] == 'Person':
                to_append = [None, None, an_dict['normalized_text'], None, an_dict['probability']]
            elif an_dict['type'] == 'Other':
                to_append = [None, None, None, an_dict['normalized_text'], an_dict['probability']]
            else:
                to_append = [None, None, None, None, None]
    annotations_df.loc[len(annotations_df.index)] = to_append

In [ ]:
entity_df.apply(lambda x: process_annotations(x['annotations']), axis=1)

In [ ]:
entity_df.apply(lambda x: process_mentions(x['mentions']), axis=1)

In [ ]:
tweet_df = tweet_df.join(annotations_df)

In [ ]:
tweet_df = tweet_df.join(mentions_df)

In [ ]:
tweet_df = tweet_df.drop(columns='entities')

In [ ]:
tweet_df = tweet_df.drop(columns='context_annotations')

In [ ]:
tweet_df

## Numerical Analysis  